In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
import sys, io
import pandas as pd 
from ipynb.fs.full.Data_Processing import *
from ipynb.fs.full.evaluation import *
from braindecode.datasets.xy import create_from_X_y
import time
import numpy as np
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet , Deep4Net
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
from collections import namedtuple
import pickle
from sklearn.model_selection import KFold

In [2]:
cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True

seed = 20200220  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed, cuda=cuda)

In [3]:
def standardise(X_train, X_valid):
    # standardize per channel
    means = X_train.mean(axis=(0,2), keepdims=True)
    stds = X_train.std(axis=(0,2), keepdims=True)
    X_train = (X_train - means) / (stds)
    X_valid = (X_valid - means) / (stds)
    return X_train, X_valid


In [13]:
def choose_cnn (cnn, trainset, validset , n_classes , device, cuda , n_epochs):
    # Extract number of chans and time steps from dataset
    n_chans = trainset[0][0].shape[0]
    input_window_samples = trainset[0][0].shape[1]

    if cnn == 'shallow':
        lr = 0.0625 * 0.01
        weight_decay = 0
        model =  ShallowFBCSPNet(n_chans, n_classes, input_window_samples=input_window_samples, final_conv_length="auto")
    else:
        lr = 1 * 0.01
        weight_decay = 0.5 * 0.001
        """
        For 30 samples, filter time_length = 1
        For 60 > samples, filter time length is left empty
        for 15 samples, filter_time length = 1, filter_length_2 = 1, filter_length_3 = 1
        """
        model =  Deep4Net(n_chans, n_classes, input_window_samples=input_window_samples,
                  final_conv_length="auto", pool_time_length=1, filter_time_length = 1,pool_time_stride=1)
    if cuda:
        model = model.cuda()

    print(next(model.parameters()).device)
    batch_size = 32

    clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(validset),  # using valid_set for validation
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,)
    return clf

In [5]:
def average(lst): 
    return sum(lst) / len(lst)

In [6]:
def categorise(y_train, y_valid):
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_valid = le.transform(y_valid)
    return y_train, y_valid, le

In [7]:
def is_slice_in_list(s,l):
    len_s = len(s) #so we don't recompute length of s on every iteration
    return any(s == l[i:len_s+i] for i in range(len(l) - len_s+1))

In [10]:
def kfold_predict (X,y, model_type, n_epochs):
    kf= KFold(n_splits = 5, shuffle = True, random_state =  1)

    if model_type == 'clf':
        results = {"Accuracy":[], "Precision":[], "Recall":[], "F1 Score Macro":[],
              "F1 Score Micro":[],"Balanced Accuracy":[]}
    else:
        results = {'RMSE':[], 'R2':[]}

    total_predictions = []
    total_true = []
    num_classes = 0
    clf = None
    for train_index, test_index in kf.split(X):
        print("Train: ", train_index, "Validation: ", test_index)

        #Train/test split
        X_train, X_valid = np.concatenate(X[train_index]), np.concatenate(X[test_index])
        y_train, y_valid = np.concatenate(y[train_index]).astype('int'), np.concatenate(y[test_index]).astype('int')
        
        y_valid_classes = list(set(y_valid))
        y_train_classes = list(set(y_train))

        if is_slice_in_list(y_valid_classes, y_train_classes) == False: continue
        print(set(y_train))
        print(set(y_valid))

        size = len(X_train) + len(X_valid) #get dataset size

        #standardise per channel
        X_train, X_valid = standardise(X_train, X_valid)
        
        #label the categorical variables 
        y_train, y_valid, le = categorise(y_train, y_valid)

        # Convert training and validation sets into a suitable format
        save_stdout = sys.stdout
        sys.stdout = open('/cs/tmp/ybk1/trash', 'w')
        trainset = create_from_X_y(X_train, y_train, drop_last_window=False)
        validset = create_from_X_y(X_valid, y_valid, drop_last_window=False)
        sys.stdout = save_stdout

        # count the number of classes
        if len(set(y_train)) > num_classes:
            num_classes = len(set(y_train))

        # commence the training process
        time_start = time.time()
        save_stdout = sys.stdout
        sys.stdout = open('/cs/tmp/ybk1/trash', 'w')
        clf = choose_cnn ('deep', trainset, validset , num_classes , device, cuda, n_epochs).fit(trainset, y=None, epochs=n_epochs)
        sys.stdout = save_stdout
        print('Training completed created! Time elapsed: {} seconds'.format(time.time()-time_start))

        # make predictions
        y_pred = le.inverse_transform(clf.predict(X_valid))
        y_true = le.inverse_transform(y_valid)
        total_predictions.append(y_pred)
        total_true.append(y_true) 
        r = get_results(y_true, y_pred, model_type)

        for key in r: # loop through dictionary to add to all the scores to the results dictionary
            results[key].append(r[key])

    for key in results: # finallly average out the results 
        results[key] = average(results[key])
    return results, np.concatenate(total_predictions), np.concatenate(total_true), num_classes, size , clf

In [14]:
time_original = time.time()

results = []
labels = ['attention','interest','effort']
window_size_samples = 30

saved_file = "/cs/home/ybk1/Dissertation/data/all_users_sampled_with_individual_tests_30_window_annotated_EEG.pickle"
all_tests = load_file(saved_file)
users = all_tests.keys()
n_epochs  = 1
model_type = 'clf'
for user in users:
    print("Working on user {0}".format(user))
    torch.backends.cudnn.benchmark = True
    
    for label in labels:
        time_start = time.time()
        dt = all_tests[user] # dictionary of all the individual tests per user

        X = np.array([np.array(x).transpose(0,2,1).astype(np.float32) for x in dt['inputs']])     
        y = np.array([np.array(x) for x in dt[label]]) #Convert the categories into labels

        # train and make predictions
        r, y_pred, y_true, num_classes, size, clf = kfold_predict(X,y, model_type, n_epochs)

         # get results
        Results = namedtuple("Results","user label n_epochs window_size time num_classes size accuracy bal_acc precision recall f1_score_macro f1_score_micro")
        results.append(Results(user, label, n_epochs, window_size_samples, time.time()-time_start,  num_classes,size, r['Accuracy'], r['Balanced Accuracy'], r['Precision'], r['Recall'], 
                              r['F1 Score Macro'], r['F1 Score Micro']))

        # plot confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        saved_file = "results/CNN/confusion/k fold/User_{0}_{1}.png".format(user,label)
        plot_confusion_matrix(cm, set(y_true), saved_file ,normalize=True)
        
        #plot loss curve
        plot_loss_curve(clf)
        plt.savefig("results/CNN/loss curves/k fold/User_{0}_{1}.png".format(user,label))
        print("Finished analysis on User {0}_{1}".format(user,label))
    print("Finished analysis on User {0}".format(user))
results  = pd.DataFrame(results).to_csv("results/CNN/tabulated/k fold/DeepCNN_Valid_performance_window_size_{0}_{1}_withclasses.csv".format(window_size_samples,label), index=False )
final_duration = time.time()- time_original
print("All analyses are complete! Time elapsed: {0}".format(final_duration))

/cs/home/ybk1/python/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


ZeroDivisionError: float division by zero

In [ ]:
# # save model 
# saved_file = "models/attention_user_1_with_fi.pickle"
# with open(saved_file, 'wb') as handle:            
#     pickle.dump(clf, handle, protocol=pickle.HIGHEST_PROTOCOL)